In [1]:
%load_ext autoreload
%autoreload 2

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
from pearce.emulator import OriginalRecipe, ExtraCrispy
from pearce.emulator import parameter, DEFAULT_PARAMS as PARAMS

In [3]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

In [4]:
training_dir = '/u/ki/swmclau2/des/PearceLHC_wp_rp'
fiducial_params = {'logM0': 12.20, 'logM1': 13.7, 'alpha': 1.02,
                      'logMmin': 12.1, 'f_c': 0.19, 'sigma_logM': 0.46}
rbins = np.array([  0.06309573,   0.12437607,   0.24517359,   0.34422476, 0.48329302, 0.67854546,\
           0.9526807 , 1.33756775,1.8779508 ,   2.6366509 ,   3.70186906,   5.19743987, 7.29722764, \
           10.24533859,  14.38449888,  20.1958975 , 28.35512583,  39.81071706] )
rpoints = (rbins[1:]+rbins[:-1])/2

In [5]:
y_param = 'logMmin'
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233)]
em_params = dict(emulation_point)
del em_params[y_param]

In [6]:
y_param_idx = -1
for idx, p in enumerate(PARAMS):
    if p.name==y_param:
        y_param_idx = idx
        break
        
y_vals = np.linspace(PARAMS[y_param_idx].low, PARAMS[y_param_idx].high, 4)
em_params[y_param] = y_vals

In [7]:
print em_params[y_param]

[ 11.7         11.96666667  12.23333333  12.5       ]


In [8]:
print len(em_params)

6


In [9]:
or_params = PARAMS[:]
or_params.append(parameter('r', 0, 1)) #95% sure bounds aren't used for r
emu1 = OriginalRecipe(training_dir,method='gbtr', or_params)

TypeError: unhashable type: 'list'

In [ ]:
emu1.get_training_data('/u/ki/swmclau2/des/PearceLHC_wp_rp_dense2', None, {})

In [ ]:
emu1.x.shape

In [ ]:
def f(x1,x2):
    return emu1.gp.kernel.value(np.array([x1]),np.array([x2]))

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
#regressor = RandomForestRegressor(n_estimators=int(1e3)) # use defaults for now
regressor = GradientBoostingRegressor(n_estimators=int(1e4), max_depth=5)
#regressor = SVR(C = 1000,kernel = emu1.gp.kernel.value)
#regressor = KernelRidge(kernel = f)
#regressor = DecisionTreeRegressor()
#regressor = MLPRegressor((10,20), learning_rate_init=0.01)

In [ ]:
emu1.x[0,:]

In [ ]:
emu1.y[0]

In [ ]:
regressor.fit(emu1.x, emu1.y)

In [ ]:
rpoints = np.logspace(np.log10(rbins[0]), np.log10(rbins[-1]), 200)

In [ ]:
print emu1.y[0]
print emu1.y_hat

In [ ]:
em_params['r'] = np.log10(rpoints)
t_list = [em_params[p.name] for p in or_params if p.name in em_params]
t_grid = np.meshgrid(*t_list)
t = np.stack(t_grid).T
        # TODO george can sort?
t = t.reshape((-1, 7))

em_point = emu1._sort_params(t)
del em_params['r']

In [ ]:
log_xi_pred = regressor.predict(em_point)
log_xi_cov = np.zeros((log_xi_pred.shape[0], log_xi_pred.shape[0]))

In [ ]:
log_xi_err = np.sqrt(np.diag(log_xi_cov))
log_xi_pred = log_xi_pred.reshape((-1, len(rpoints)))
log_xi_err = log_xi_err.reshape((-1, len(rpoints)))

In [ ]:
truth_dir = '/u/ki/swmclau2/des/PearceFHC_wp_rp/'
plot_r, plot_xi, plot_xi_err = emu1.get_plot_data(em_params,truth_dir, independent_variable=None)

In [ ]:
plot_xi.shape, plot_r.shape, plot_xi_err.shape

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, lxp, lxe,pr, px, err,c in zip(y_vals, log_xi_pred, log_xi_err ,plot_r, plot_xi,plot_xi_err,colors):

    plt.plot(rpoints, lxp,label = '%.3f'%label, color = c )

    plt.fill_between(rpoints,lxp+lxe,
                     lxp-lxe, color = c, alpha = 0.3)

    plt.errorbar(10**pr, px,err, color =c,fmt = 'o')

plt.xlim(xmin = 0.1, xmax = 30)    
plt.ylim(ymin = -2, ymax = 4)
plt.title("OR Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{w_p(r_p) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

In [ ]:
from glob import glob
from os import path
truth_dir = '/u/ki/swmclau2/des/PearceFHC_wp_rp'
N=None
statistic = 'rel'
cov_files = glob(path.join(truth_dir, 'cov*.npy'))
obs_files = glob(path.join(truth_dir, 'obs*.npy'))

In [ ]:
from pearce.emulator import GLOBAL_FILENAME, global_file_reader, obs_file_reader
from time import time

In [ ]:
np.random.seed(int(time()))

if N is None:
    idxs = np.arange(len(obs_files))
else:
    idxs = np.random.choice(len(obs_files), N, replace=False)

values = []
bins, _, _, _ = global_file_reader(path.join(truth_dir, GLOBAL_FILENAME))
bin_centers = (bins[1:] + bins[:-1]) / 2
for idx in idxs:
    params, true_obs, _ = obs_file_reader(obs_files[idx], cov_files[idx])
    #pred_log_obs, _ = self.emulate_wrt_r(params, bin_centers)
    params['r'] = np.log10(bin_centers)
    t_list = [params[p.name] for p in or_params if p.name in params]
    t_grid = np.meshgrid(*t_list)
    t = np.stack(t_grid).T
    # TODO george can sort?
    t = t.reshape((-1, 7))

    em_point = emu1._sort_params(t)
    del params['r']
    
    pred_log_obs = regressor.predict(em_point)

    if statistic == 'rmsfd':
        values.append(np.sqrt(np.mean(((pred_log_obs - np.log10(true_obs)) ** 2) / (np.log10(true_obs) ** 2))))
    elif statistic == 'r2':  # r2
        SSR = np.sum((pred_log_obs - np.log10(true_obs)) ** 2)
        SST = np.sum((np.log10(true_obs) - np.log10(true_obs).mean()) ** 2)

        values.append(1 - SSR / SST)

    elif statistic == 'abs':
        values.append((pred_log_obs - np.log10(true_obs)) )
        #values.append((10**pred_log_obs - true_obs) )
    else: #'rel'
        values.append(((pred_log_obs - np.log10(true_obs)) )/np.log10(true_obs) )
        #values.append((10**pred_log_obs - true_obs )/true_obs )
values = np.array(values)

In [ ]:
gbt_err = np.abs(values).mean(axis=0)*100

In [ ]:
print gbt_err

In [ ]:
#LHC wp dense
gp_err = np.array([0.96249449,0.70766422,1.09460978, 0.76991548, 0.84404117, 0.80792512, 0.84134409, 0.79983003,
          0.86213437,1.03998899,1.04112086,1.20671371,1.56148868,2.18618773,3.46016119,14.03835135,  163.17511754])

In [ ]:
gbt_err = gbt_err/100
gp_err = gp_err/100

In [ ]:
fig = plt.figure(figsize = (10, 10))
plt.plot(bin_centers, gbt_err, label='Gradient Boosted Trees')
plt.plot(bin_centers, gp_err, label = 'Gaussian Process')

plt.xlim(xmin = 0.1, xmax = 30)
plt.ylim(ymin = 0, ymax = 0.05)
plt.title(r"Rel err $w_p(r_p)$ different methods", fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'Relative Error', fontsize = 25)
plt.legend(loc='best')

In [ ]:
%%bash
ls *.npy

In [ ]:
box = 'fhc'
gbt_vals = np.loadtxt('gbtr_%s.npy'%box)
gbt_err = np.abs(gbt_vals).mean(axis=0)
gbt_dense_vals = np.loadtxt('gbtr_dense_%s.npy'%box)
gbt_dense_err = np.abs(gbt_dense_vals).mean(axis=0)
svr_vals = np.loadtxt('svr_%s.npy'%box)
svr_err = np.abs(svr_vals).mean(axis=0)
gp_vals = np.loadtxt('gp_%s.npy'%box)
gp_err = np.abs(gp_vals).mean(axis=0)

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, err in zip(('GBT','GBT More Training', 'SVR', 'GP'), (gbt_err,gbt_dense_err, svr_err, gp_err)):
    plt.plot(bin_centers, err, label=label)

plt.xlim(xmin = 0.1, xmax = 30)
plt.ylim(ymin = 0, ymax = 0.05)
plt.title(r"Rel err $w_p(r_p)$ different methods %s"%box, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'Relative Error', fontsize = 25)
plt.legend(loc='best')